In [1]:
import pandas as pd
import numpy as np

In [2]:
lists = ['apple']*3 + ['orange']*2 + ['banana']*2
fruits = pd.Series(lists)
fruits

0     apple
1     apple
2     apple
3    orange
4    orange
5    banana
6    banana
dtype: object

Berikut adalah frekuensi relatif setiap buah dalam keranjang, yang dapat dianggap sebagai distribusi probabilitas buah.

In [3]:
probs = fruits.value_counts(normalize=True)
probs

apple     0.428571
banana    0.285714
orange    0.285714
dtype: float64

Jika Anda suka, Anda dapat menentukan sendiri distribusi probabilitas seperti di bawah ini.

In [4]:
probs_by_hand = round(3/7, 6), round(2/7, 6), round(2/7, 6)
probs_by_hand

(0.428571, 0.285714, 0.285714)

Ingatlah bahwa model Shannon mendefinisikan entropi sebagai



Gagasan dengan entropi adalah bahwa semakin heterogen dan tidak murni suatu fitur, semakin tinggi entropi tersebut. Sebaliknya, semakin homogen dan murni suatu ciri, semakin rendah entropinya. Perhitungan berikut menunjukkan bagaimana pengotor sekeranjang buah ini dapat dihitung menggunakan kriteria entropi.

In [5]:
entropy = -1 * np.sum(np.log2(probs) * probs)
entropy

1.5566567074628228

Sebagai perbandingan, mari kita hitung ketidakmurnian sekeranjang buah lain dengan 7 buah berbeda dengan frekuensi yang sama.

In [6]:
lst2 = ['apple', 'orange', 'banana', 'mango', 'blueberry', 'watermelon', 'pear']
fruits2 = pd.Series(lst2)
fruits2

0         apple
1        orange
2        banana
3         mango
4     blueberry
5    watermelon
6          pear
dtype: object

In [7]:
probs2 = fruits2.value_counts(normalize=True)
probs2

blueberry     0.142857
apple         0.142857
mango         0.142857
orange        0.142857
pear          0.142857
banana        0.142857
watermelon    0.142857
dtype: float64

In [8]:
entropy = -1 * np.sum(np.log2(probs2) * probs2)
entropy

2.807354922057604

## Play Tennis Example

Kami sekarang mengerjakan detail kalkulasi ketidakmurnian untuk dataset vegetation. Mari pertama-tama impor dataset dari cloud.

In [9]:
df = pd.read_csv("dataset/play_tennis.csv", index_col=['day'])
df

,outlook,temp,humidity,wind,play
day,,,,,
D1,Sunny,Hot,High,Weak,No
D2,Sunny,Hot,High,Strong,No
D3,Overcast,Hot,High,Weak,Yes
D4,Rain,Mild,High,Weak,Yes
D5,Rain,Cool,Normal,Weak,Yes
D6,Rain,Cool,Normal,Strong,No
D7,Overcast,Cool,Normal,Strong,Yes
D8,Sunny,Mild,High,Weak,No
D9,Sunny,Cool,Normal,Weak,Yes


In [10]:
def compute_impurity(feature):
    probs = feature.value_counts(normalize=True)
    impurity = -1 * np.sum(np.log2(probs) * probs)
    return(round(impurity, 3))

print('impurity using entropy:', compute_impurity(fruits))

impurity using entropy: 1.557


Mari menghitung entropi dari fitur target "play" menggunakan fungsi baru kita.

In [11]:
target_entropy = compute_impurity(df['play'])
target_entropy

0.94

Mari menghitung information gain berdasarkan fitur deskriptif untuk mengetahui fitur terbaik. Untuk tugas ini, kami melakukan hal berikut:
1. Hitung ketidakmurnian fitur target (menggunakan indeks entropi).
2. Partisi kumpulan data berdasarkan nilai unik dari fitur deskriptif.
3. Hitung ketidakmurnian untuk setiap partisi.
4. Hitung sisa ketidakmurnian sebagai jumlah tertimbang dari ketidakmurnian setiap partisi.
5. Hitung perolehan informasi sebagai perbedaan antara ketidakmurnian fitur target dan ketidakmurnian yang tersisa.

Kami akan mendefinisikan fungsi lain untuk mencapai ini, yang disebut comp_feature_information_gain(). Sebagai contoh, mari kita lihat level dari fitur "wind"

In [12]:
df['wind'].value_counts()

Weak      8
Strong    6
Name: wind, dtype: int64

Mari kita lihat bagaimana tampilan partisi untuk fitur ini dan apa penghitungan yang sesuai menggunakan kriteria pemisahan entropi.

In [13]:
for level in df['wind'].unique():
    print('level name:', level)
    df_feature_level = df[df['wind'] == level]
    print('corresponding data partition:')
    print(df_feature_level['play'])
    print('partition target feature impurity:', compute_impurity(df_feature_level['play']))
    print('partition weight:', str(len(df_feature_level)) + '/' + str(len(df)))
    print('====================')

level name: Weak
corresponding data partition:
day
D1      No
D3     Yes
D4     Yes
D5     Yes
D8      No
D9     Yes
D10    Yes
D13    Yes
Name: play, dtype: object
partition target feature impurity: 0.811
partition weight: 8/14
level name: Strong
corresponding data partition:
day
D2      No
D6      No
D7     Yes
D11    Yes
D12    Yes
D14     No
Name: play, dtype: object
partition target feature impurity: 1.0
partition weight: 6/14


Idenya di sini adalah, untuk masing-masing dari 4 partisi data di atas
1. Kami menghitung ketidakmurniannya sehubungan dengan fitur target sebagai kumpulan data yang berdiri sendiri.
2. Kami menimbang ketidakmurnian ini dengan jumlah pengamatan relatif di setiap partisi. Jumlah pengamatan relatif dihitung sebagai jumlah pengamatan di partisi dibagi dengan jumlah pengamatan di seluruh kumpulan data. Misalnya, berat partisi pertama adalah 3/7.
3. Kami menjumlahkan pengotor tertimbang ini dan menyebutnya sebagai pengotor yang tersisa untuk fitur ini.

Misalnya, sisa pengotor yang diukur dengan entropi untuk fitur elevasi adalah 0,918 x (3/7) + 1,0 x (2/7) = 0,679. Keuntungan informasi kemudian dihitung sebagai 1,557 - 0,679 = 0,878. Sekarang kita siap untuk mendefinisikan fungsi kita. Ada sedikit pengkodean di sini, tetapi kami dapat meyakinkan Anda bahwa mencoba mencari tahu cara kerja di sini akan bermanfaat untuk meningkatkan keterampilan pemrograman Python Anda.

In [17]:
def comp_feature_information_gain(df, target, descriptive_feature):    
    entropy_list = list()
    weight_list = list()
    
    for level in df[descriptive_feature].unique():
        df_feature_level = df[df[descriptive_feature] == level]
        entropy_level = compute_impurity(df_feature_level[target])
        entropy_list.append(round(entropy_level, 3))
        weight_level = len(df_feature_level) / len(df)
        weight_list.append(round(weight_level, 3))
        

    target_entropy = compute_impurity(df[target])
    feature_remaining_impurity = np.sum(np.array(entropy_list) * np.array(weight_list))
    
    information_gain = target_entropy - feature_remaining_impurity

    return(information_gain, descriptive_feature)

def information_gain(dataset, target):
    information_gain = []
    
    for feature in dataset.drop(columns=target).columns:
        information_gain.append(comp_feature_information_gain(dataset, target, feature))
        
    result = pd.DataFrame(
        information_gain, 
        columns=['information gain', 'feature']
    ).sort_values(
        'information gain',
        ascending=False
    )
    return result

In [18]:
information_gain(df, 'play')

,information gain,feature
0,0.246706,outlook
2,0.151500,humidity
3,0.047919,wind
1,0.028232,temp
